In [ ]:
from devtools import debug
from dotenv import load_dotenv

%load_ext autoreload
%autoreload 2

load_dotenv()
!export PYTHONPATH=":./python"

In [ ]:
# cSpell:disable

from devtools import debug
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


class Parcours(BaseModel):
    metiers: list[str] = Field(
        description="liste de metiers auquels peut conduire la formation"
    )
    domaines: list[str] = Field(
        description="domaines auquels s'applique cette formation"
    )


system = """
    Extraire les metiers et les domaines d'une fiche de description de formation récupérée sur internet.  
    Repondre seulement avec les informations disponibles dans la description.
    \n
    """
user = """
    fiche de desctiption de formation:
    --- 
    {fiche}
    --- 
    {format_instructions}"""

parser = PydanticOutputParser(pydantic_object=Parcours)

prompt = def_prompt(system=system, user=user).partial(
    format_instructions=parser.get_format_instructions()
)

llm = get_llm(temperature=0.5)
chain = prompt | llm | parser
summaries = chain.batch([{"fiche": page_cleaned}], {"max_concurrency": 5})

debug(summaries)